# Setup

In [1]:
%load_ext lab_black

In [2]:
# wrap output text
from IPython.display import HTML, display


def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )


get_ipython().events.register("pre_run_cell", set_css)

In [3]:
import torch

# Check if CUDA (GPU support) is available
cuda_available = torch.cuda.is_available()

# Get the PyTorch version
pytorch_version = torch.__version__

# Get information about the GPU if available
gpu_info = None
if cuda_available:
    current_device = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(current_device)
    gpu_info = f"Using GPU: {device_name} (Device {current_device})"

# Print the information
print(f"PyTorch Version: {pytorch_version}")
if cuda_available:
    print("CUDA (GPU) is available.")
    print(gpu_info)
else:
    print("CUDA (GPU) is not available. Running on CPU.")

PyTorch Version: 2.0.1
CUDA (GPU) is not available. Running on CPU.


In [4]:
import os
import textwrap
import numpy as np

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# load data

In [5]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

# dataset = load_dataset("samsum")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# view data


In [6]:
print(f"train num rows : {len(dataset['train'])}")
print(f"validation num rows : {len(dataset['validation'])}")
print(f"test num rows : {len(dataset['test'])}")

train num rows : 12460
validation num rows : 500
test num rows : 1500


In [7]:
print_dashes = lambda: print("-" * 80)
txtwrap = lambda text: textwrap.fill(text, width=80)
print_dialogue = lambda x: [print(txtwrap(text=i)) for i in x.split("\n")]

In [8]:
view_data_ix = [10, 12]

print_dashes()
for i in view_data_ix:
    print(f"index__{i}")
    print_dashes()
    print(f"DIALOGUE:")
    print_dialogue(dataset["train"][i]["dialogue"])
    print(f"\nSUMMARY:")
    print_dialogue(dataset["train"][i]["summary"])
    print_dashes()

--------------------------------------------------------------------------------
index__10
--------------------------------------------------------------------------------
DIALOGUE:
#Person1#: Could you do me a favor?
#Person2#: Sure. What is it?
#Person1#: Could you run over to the store? We need a few things.
#Person2#: All right. What do you want me to get?
#Person1#: Well, could you pick up some sugar?
#Person2#: Okay. How much?
#Person1#: A small bag. I guess we also need a few oranges.
#Person2#: How many?
#Person1#: Oh, let's see. . . About six.
#Person2#: Anything else?
#Person1#: Yes. We're out of milk.
#Person2#: Okay. How much do you want me to get? A gallon?
#Person1#: No. I think a half gallon will be enough.
#Person2#: Is that all?
#Person1#: I think so. Have you got all that?
#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of
milk.
#Person1#: Do you have enough money?
#Person2#: I think so.
#Person1#: Thanks very much. I appreciate it.

SUMMARY

# load model

In [9]:
model_id = "google/flan-t5-base"

# Load tokenizer of FLAN-t5-small
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [10]:
def baseline_summary_predictions(dialogue, summary, tokenizer, model):
    enc = tokenizer.encode(dialogue, return_tensors="pt")
    print_dashes()
    print("DIALOGUE:")
    print_dialogue(dialogue)
    print()
    print("ACTUAL SUMMARY:")
    print(txtwrap(summary))
    print()

    model_out = model.generate(enc, max_new_tokens=100)[0]
    model_summary = tokenizer.decode(model_out, skip_special_tokens=True)

    print("BASELINE MODEL SUMMARY")
    print(txtwrap(model_summary))
    print_dashes()


view_data_ix = [140, 100]


for i in view_data_ix:
    baseline_summary_predictions(
        dataset["train"][i]["dialogue"],
        dataset["train"][i]["summary"],
        tokenizer=tokenizer,
        model=model,
    )

--------------------------------------------------------------------------------
DIALOGUE:
#Person1#: Um, which of them is the better typist?
#Person2#: Well, Mary types faster than Jones. But I think Jones types more
carefully.
#Person1#: Is there any difference in their short hand?
#Person2#: Jones can certainly take down letter more quickly, but Mary's short
hand is the best in her class.
#Person1#: How about languages?
#Person2#: Both of them speak excellent French. But I think Mary speaks German
more fluently than Jones does.
#Person1#: Um. That doesn't really matter. Because we have more French visitors
than German.

ACTUAL SUMMARY:
#Person1# asks #Person2# to compare Mary and Jones in terms of typing,
shorthand, and language mastery.

BASELINE MODEL SUMMARY
People are more French than German.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DIALOGUE:
#Person1#: I have

In [11]:
# dummy_sent = "hello bob how are you doing today, how is the weather hellloo helloo theree how are you"
# dummy_sent_enc = tokenizer.encode(dummy_sent, return_tensors="pt")
# print(dummy_sent_enc)
# print(tokenizer.decode(dummy_sent_enc[0], skip_special_tokens=True))
# model.generate(dummy_sent_enc, max_new_tokens=100)

# Prompts and outputs


## Zero shot inference with instruction

In [12]:
def zero_shot(dialogue, summary, prompt, tokenizer, model):
    print_dashes()
    # print("ORIGNAL DIALOGUE:")
    # print_dialogue(dialogue)
    # print()

    print("PROMPT")
    new_prompt = prompt(dialogue)
    print_dialogue(new_prompt)
    enc = tokenizer.encode(new_prompt, return_tensors="pt")
    print()
    model_out = model.generate(enc, max_new_tokens=100)[0]
    model_summary = tokenizer.decode(model_out, skip_special_tokens=True)

    print("BASELINE MODEL SUMMARY")
    print(txtwrap(model_summary))
    print()
    print("ACTUAL SUMMARY:")
    print(txtwrap(summary))
    print()
    print_dashes()


prompt_map = dict(
    p1=(
        lambda d: f"""
Dialogue -
{d}

Summary - ?
"""
    ),
    p2=lambda d: f"""
Summarize the conversation below:
{d}
""",
    p3=lambda d: f"""
Explain the conversation:
{d}
""",
    p4=lambda d: f"""
Dialogue:
{d}

What is going on in the conversation here?
""",
)


query_ix = [140, 100]


for i in query_ix:
    zero_shot(
        dialogue=dataset["train"][i]["dialogue"],
        summary=dataset["train"][i]["summary"],
        prompt=prompt_map["p4"],
        tokenizer=tokenizer,
        model=model,
    )

--------------------------------------------------------------------------------
PROMPT

Dialogue:
#Person1#: Um, which of them is the better typist?
#Person2#: Well, Mary types faster than Jones. But I think Jones types more
carefully.
#Person1#: Is there any difference in their short hand?
#Person2#: Jones can certainly take down letter more quickly, but Mary's short
hand is the best in her class.
#Person1#: How about languages?
#Person2#: Both of them speak excellent French. But I think Mary speaks German
more fluently than Jones does.
#Person1#: Um. That doesn't really matter. Because we have more French visitors
than German.

What is going on in the conversation here?


BASELINE MODEL SUMMARY
Mary is the better typist.

ACTUAL SUMMARY:
#Person1# asks #Person2# to compare Mary and Jones in terms of typing,
shorthand, and language mastery.

--------------------------------------------------------------------------------
---------------------------------------------------------------

## Few Shot Inference

In [13]:
def few_shot(query, summary, tokenizer, model):
    print_dashes()
    # print("ORIGNAL DIALOGUE:")
    # print_dialogue(dialogue)
    # print()

    print("PROMPT")
    print(query)
    enc = tokenizer.encode(query, return_tensors="pt")
    print()
    model_out = model.generate(
        enc,
        max_new_tokens=200,
    )[0]
    model_summary = tokenizer.decode(model_out, skip_special_tokens=True)

    print("#" * 30)
    print("BASELINE MODEL SUMMARY")
    print(txtwrap(model_summary))
    print()
    print("ACTUAL SUMMARY:")
    print(txtwrap(summary))
    print("#" * 30)
    print()
    print_dashes()


def make_prompt(query, example_ix, dataset):
    p = ""
    for i in example_ix:
        d = dataset["train"][i]["dialogue"]
        s = dataset["train"][i]["summary"]
        p += f"""DIALOGUE:\n{d}\n\nSUMMARY OF DIALOGUE:\n{s}\n\n"""

    p = f"""{p}DIALOGUE:\n{query}\n\nSUMMARY OF DIALOGUE:?\n\n"""
    # print_dialogue(p)
    return p


# make_prompt(dataset['train'][16]['dialogue'],[30], dataset)

### single shot inference

In [14]:
query_ix = [160, 210, 340]

for i in query_ix:
    q = dataset["train"][i]["dialogue"]
    s = dataset["train"][i]["summary"]
    few_shot_prompt = make_prompt(q, [100], dataset)

    few_shot(
        query=few_shot_prompt,
        summary=s,
        tokenizer=tokenizer,
        model=model,
    )

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


--------------------------------------------------------------------------------
PROMPT
DIALOGUE:
#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.

SUMMARY OF DIALOGUE:
#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.

DIALOGUE:
#Person1#: Come on in. I am so glad you are here. We have been expecting you.
#Person2#: Am I late for the party? Looks everyone is al

## two shot

In [15]:
query_ix = [160, 210, 340]

for i in query_ix:
    q = dataset["train"][i]["dialogue"]
    s = dataset["train"][i]["summary"]
    few_shot_prompt = make_prompt(q, [100, 700], dataset)

    few_shot(
        query=few_shot_prompt,
        summary=s,
        tokenizer=tokenizer,
        model=model,
    )

--------------------------------------------------------------------------------
PROMPT
DIALOGUE:
#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.

SUMMARY OF DIALOGUE:
#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.

DIALOGUE:
#Person1#: Good evening, sir. What's your pleasure?
#Person2#: Give me a shot of whisky and soda.
#Person1#: With or without ice, sir?

- We can see that even with two shot inference the results did not improve.
- This model needs further fine tuning if we need to summarize the conversations